In [1]:
import pandas as pd
import os
import shutil
import subprocess

In [2]:
import pandas as pd

file_path = 'PN03-1_data.txt'

# Carregar a primeira linha como cabeçalho
df_header = pd.read_csv(file_path, nrows=1)

# Definir o intervalo de linhas a serem lidas
start_row = 19857406 - 500000  # Linha inicial (após o cabeçalho, 0-indexed)
end_row = 19857406 + 500000    # Linha final (linha a ser lida, inclusive)

# Carregar o intervalo específico de linhas
df = pd.read_csv(file_path, skiprows=range(1, start_row), nrows=end_row - start_row + 1, header=None)

# Definir as colunas do DataFrame com base no cabeçalho
df.columns = df_header.columns

print(df)

C:\Users\orlando\AppData\Local\Temp\ipykernel_4720\2045892482.py:13: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, skiprows=range(1, start_row), nrows=end_row - start_row + 1, header=None)


                 Time        1      2       3      4       5       6       7  \
0                Time    1.000  2.000   3.000  4.000   5.000   6.000   7.000   
1        37807.431641  -67.500  0.000  -7.750 -0.250  -6.625  -6.500 -10.750   
2        37807.433594   -6.625  1.125 -16.250 -1.375  -7.625  -3.875  -8.000   
3        37807.435547  -60.625  1.000  -1.750 -1.750  -8.250  -2.500  -5.875   
4        37807.437500  219.750  0.625  93.500  0.125  -6.250  -0.375   0.750   
...               ...      ...    ...     ...    ...     ...     ...     ...   
999996   39760.546875  -17.625 -0.875  22.750  5.000  61.875  14.375  38.500   
999997   39760.548828  -15.750  0.250  13.500  5.125  62.500  15.875  40.500   
999998   39760.550781  -15.625  0.000   9.500  5.000  62.500  15.125  49.750   
999999   39760.552734  -16.125 -0.625 -24.000  3.750  61.750  14.750  49.375   
1000000  39760.554688  -16.750 -0.250 -26.125  3.375  60.625  19.500  52.750   

               8        9  ...       26

General names
- 03_Fp1_General  BLACK
- 03_O1_General   RED
- 03_Cz_General   GREEN
- 03_Fp2_General  BLUE
- 03_O2_General   Cyan

Pre-ictal names
- 03_Fp1_Pre-ictal  BLACK
- 03_O1_Pre-ictal   RED
- 03_Cz_Pre-ictal   GREEN
- 03_Fp2_Pre-ictal  BLUE
- 03_O2_Pre-ictal   Cyan

Ictal names
- 03_Fp1_Ictal      BLACK
- 03_O1_Ictal       RED
- 03_Cz_Ictal       GREEN
- 03_Fp2_Ictal      BLUE
- 03_O2_Ictal       Cyan

Post-ictal names
- 03_Fp1_Post-ictal  BLACK
- 03_O1_Post-ictal   RED
- 03_Cz_Post-ictal   GREEN
- 03_Fp2_Post-ictal  BLUE
- 03_O2_Post-ictal   Cyan

In [3]:
channels_columns = [
    ['1', 'C1'],   # B -> channel 1,  Fp1 Black
    ['5', 'C5'],   # F -> channel 5,  O1 Red
    ['15', 'C15'], # P -> channel 15, Cz Green
    ['17', 'C17'], # R -> channel 17, Fp2 Blue
    ['21', 'C21']  # V -> channel 21  O2 Cyan
]

In [4]:
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split('.'))
    return h * 3600 + m * 60 + s

In [5]:
#File name: PN03-1.edf
#Registration start time:22.44.37
#Registration end time: 11.41.34
#Seizure start time:09.29.10
#Seizure end time: 09.31.01

## CORRECTED
#Registration start time:00.44.37
#Registration end time: 13.41.34
#Seizure start time:11.29.10
#Seizure end time: 11.31.01

registration_start_time_sec = time_to_seconds('00.44.37')
seizure_start_time_sec = time_to_seconds('11.29.10')
seizure_end_time_sec = time_to_seconds('11.31.01')

registration_start_time_sec = 19857406 + 500000

start_line_ictal = seizure_start_time_sec * 512 - registration_start_time_sec - 2 # Taking the first two lines out of the count
end_line_ictal =   seizure_end_time_sec * 512 - registration_start_time_sec - 2 # Taking the first two lines out of the count

# Show the results
print(f"Start Line Ictal: {start_line_ictal}")
print(f"End Line Ictal: {end_line_ictal}")

Start Line Ictal: 813792
End Line Ictal: 870624


## Cutting the ICTAL series

In [6]:
result_folder = 'ICTAL'
if os.path.exists(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

for column, channel in channels_columns:
    current_section = df.loc[start_line_ictal:end_line_ictal, column]
    values = current_section.values
    final_file_path = os.path.join(result_folder, channel + '.txt')
    with open(final_file_path, 'w') as f:
        for value in values:
            f.write(f"{value}\n")

values [ -92.125 -102.    -113.375 ...  -10.75   -10.     -13.625]
values [ 3.     0.375  2.5   ... 42.25  42.5   49.875]
values [-10.125 -19.5   -23.625 ...  24.375  32.75   36.125]
values [-43.75  -42.    -49.375 ...  81.75   73.5    70.75 ]
values [ 47.875  27.875  25.625 ...   5.75  -22.625 -19.375]


## Cutting the GENERAL series

In [7]:
start_line_general = (start_line_ictal + end_line_ictal)/2 - 76800 # Calculating the general starting point based on 0.2.30 (2 minutes and 30 seconds) before the ictal starts
end_line_general = (start_line_ictal + end_line_ictal)/2 + 76800   # Calculating the general ending point based on 0.2.30 (2 minutes and 30 seconds) after the ictal ends

result_folder = 'GENERAL'
if os.path.exists(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

for column, channel in channels_columns:
    current_section = df.loc[start_line_general:end_line_general, column]
    values = current_section.values
    
    final_file_path = os.path.join(result_folder, channel + '.txt')
    with open(final_file_path, 'w') as f:
        for value in values:
            f.write(f"{value}\n")

## Cutting the PRE ICTAL series

In [8]:
start_line_pre_ictal = start_line_ictal - (abs(start_line_ictal - end_line_ictal))  # Calculating the start of the pre-ictal based on the start of the ictal
end_line_pre_ictal = start_line_ictal 

result_folder = 'PRE_ICTAL'
if os.path.exists(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

for column, channel in channels_columns:
    current_section = df.loc[start_line_pre_ictal:end_line_pre_ictal, column]
    values = current_section.values
    
    final_file_path = os.path.join(result_folder, channel + '.txt')
    with open(final_file_path, 'w') as f:
        for value in values:
            f.write(f"{value}\n")

## Cutting the POST ICTAL series

In [9]:
start_line_post_ictal = end_line_ictal 
end_line_post_ictal = end_line_ictal + (abs(start_line_ictal - end_line_ictal)) # Calculating the end of the post-ictal based on the end of the ictal

result_folder = 'POST_ICTAL'
if os.path.exists(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

for column, channel in channels_columns:
    current_section = df.loc[start_line_post_ictal:end_line_post_ictal, column]
    values = current_section.values
    
    final_file_path = os.path.join(result_folder, channel + '.txt')
    with open(final_file_path, 'w') as f:
        for value in values:
            f.write(f"{value}\n")

# Running DFA

In [10]:
def run_dfa_exe(directory, input_files, output_files):
    dfa_exe_path = 'dfa12.exe'
    
    command = [dfa_exe_path]
    
    input_str = (
        f"{input_files[0]}\n"
        f"{input_files[1]}\n"
        f"{output_files[0]}\n"
        f"{output_files[1]}\n"
    )
    
    process = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate(input=input_str)
    
    if process.returncode != 0:
        print(f"Erro ao executar dfa12.exe na pasta {directory}: {stderr}")
    else:
        print(f"Saída do dfa12.exe na pasta {directory}:\n{stdout}")

def process_directory(directory):
    files = [f for f in os.listdir(directory) if f.endswith('.txt')]
    files.sort() 

    for i in range(0, len(files), 2):
        if i + 1 < len(files):
            input_files = [os.path.join(directory, files[i]), os.path.join(directory, files[i + 1])]
            output_files = [os.path.join(directory, files[i].replace('.txt', '.dat')), os.path.join(directory, files[i + 1].replace('.txt', '.dat'))]

            print("input_files a:", input_files)
            print("output_files a:", output_files)
            run_dfa_exe(directory, input_files, output_files)
        else:
            input_files = [os.path.join(directory, files[i]), os.path.join(directory, files[i])]
            output_files = [os.path.join(directory, files[i].replace('.txt', '.dat'))] * 2

            print("input_files b:", input_files)
            print("output_files b:", output_files)
            run_dfa_exe(directory, input_files, output_files)

directories = ['GENERAL', 'ICTAL', 'PRE_ICTAL', 'POST_ICTAL']

for directory in directories:
    if os.path.exists(directory):
        process_directory(directory)
    else:
        print(f"Pasta {directory} não encontrada.")


input_files a: ['GENERAL\\C1.txt', 'GENERAL\\C15.txt']
output_files a: ['GENERAL\\C1.dat', 'GENERAL\\C15.dat']
Saída do dfa12.exe na pasta GENERAL:
Entre com o nome da primeira serie a ser lida:
Entre com o nome da segunda serie a ser lida:
(....com o mesmo numero de pontos N):
Qual o nome do primeiro arquivo de saida (para o DFA1)?
Qual o nome do segundo arquivo de saida (para o DFA2)?

input_files a: ['GENERAL\\C17.txt', 'GENERAL\\C21.txt']
output_files a: ['GENERAL\\C17.dat', 'GENERAL\\C21.dat']
Saída do dfa12.exe na pasta GENERAL:
Entre com o nome da primeira serie a ser lida:
Entre com o nome da segunda serie a ser lida:
(....com o mesmo numero de pontos N):
Qual o nome do primeiro arquivo de saida (para o DFA1)?
Qual o nome do segundo arquivo de saida (para o DFA2)?

input_files b: ['GENERAL\\C5.txt', 'GENERAL\\C5.txt']
output_files b: ['GENERAL\\C5.dat', 'GENERAL\\C5.dat']
Saída do dfa12.exe na pasta GENERAL:
Entre com o nome da primeira serie a ser lida:
Entre com o nome da seg